In [ ]:
f# Here we are builing an Image Classfier using Keras. The dataset used is from Kaggle Dog vs Cat Image 
# Classification Problem. First let's build the dataset
# Step 1: Download the dataset from Kaggle
# Step 2: Unzip the dataset
# Step 3: You will find 2 folder, test and train
# Step 4: Delete the test folder. We will create our own test folder.
# Step 5: Inside both the train and test folders, create 2 sub-folders cats and dogs.
# Step 7: Put all the cat's image in cats folder and all the dog's image in dogs folder
# Step 8: Take some images (I took 2000) from train>cats folder and put in test>cats folder
# Step 9: Take some images (I took 2000) from train>dogs folder and put in test>dogs folder
# Step 10: Your dataset is ready

In [69]:
from keras.models import Sequential# Import the sequential layer. 
#Generally there are two types of layers, sequential and functional. Sequential is most common one
from keras.layers import Conv2D,Activation,MaxPooling2D,Dense,Flatten,Dropout
import numpy as np

In [ ]:
#Initialize a catDogImageclassifier variable here

In [70]:
catDogImageclassifier = Sequential()

In [ ]:
# We are adding layers to our network here.
# Conv2D: 2 dimensional convolutional layer
# 32: filters required. 
# 3,3: size of the filter (3 rows, 3 columns)
# Input Image shape is 64*64*3 - height*width*RGB. Each number represents pixel intensity (0-255)


In [71]:
catDogImageclassifier.add(Conv2D(32,(3,3),input_shape=(64,64,3)))

In [ ]:
# Output is a feature map. The training data will work on it and get some feature maps

In [ ]:
# Lets add the activation function now. We are using ReLU (Rectified Linear Unit). 
#The activation function gives the output basis the output. In the feature map output from the previous layer,
#the activation function will replace all the negative pixels with zero

In [72]:
catDogImageclassifier.add(Activation('relu'))

In [ ]:
# We do not want out network to be overly complex computationally, hence the pooling layer comes into picture
# The pooling layer will reduce the dimensions. Max with two by two filter, will take the maximum value but the 
# significant features will be retained

In [73]:
catDogImageclassifier.add(MaxPooling2D(pool_size =(2,2)))

In [ ]:
# All three convolutional blocks. 

In [74]:
catDogImageclassifier.add(Conv2D(32,(3,3))) # Convolutional Layer
catDogImageclassifier.add(Activation('relu')) # Activation Layer
catDogImageclassifier.add(MaxPooling2D(pool_size =(2,2))) # Pooling Layer
catDogImageclassifier.add(Conv2D(32,(3,3))) # Convolutional Layer
catDogImageclassifier.add(Activation('relu')) # Activation Layer
catDogImageclassifier.add(MaxPooling2D(pool_size =(2,2))) # Pooling Layer
catDogImageclassifier.add(Conv2D(32,(3,3))) # Convolutional Layer
catDogImageclassifier.add(Activation('relu')) # Activation Layer
catDogImageclassifier.add(MaxPooling2D(pool_size =(2,2))) # Pooling Layer

In [ ]:
# Overfitting is a nuicance. We have to fight it using Drop out. Prepare the data by flatenning it. 
#And flattening to 1 dimension

In [75]:
catDogImageclassifier.add(Flatten())

In [ ]:
# Add dense function now followed by ReLU activation

In [76]:
catDogImageclassifier.add(Dense(64))
catDogImageclassifier.add(Activation('relu'))

In [ ]:
# Here add the doropout layer
# Overfitting means that model is working good for training but failing on testing dataset

In [77]:
catDogImageclassifier.add(Dropout(0.5))

In [ ]:
# Add one more fully connected layer now to get the output in n-dimensional classes (a vector will be the output)

In [78]:
catDogImageclassifier.add(Dense(1))

In [ ]:
# Sigmoid function to convert to probabilities

In [79]:
catDogImageclassifier.add(Activation('sigmoid'))

In [ ]:
# Let us look how out network looks

In [80]:
catDogImageclassifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
activation_22 (Activation)   (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 29, 29, 32)        9248      
_________________________________________________________________
activation_23 (Activation)   (None, 29, 29, 32)        0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 12, 12, 32)        9248      
__________

In [ ]:
# A quick look at the network summary states that total number of parameters in our network are 36,961. 
#Play around with different network structures and have a look how this number changes

In [81]:
catDogImageclassifier.compile(optimizer ='rmsprop',# rmsprop is the optimizer using Gradient Descent
                   loss ='binary_crossentropy', # Loss or cost function for the model
                   metrics =['accuracy']) # The KPI 

In [ ]:
# Let us do some data augmentation here. It helps to fight overfitting. Zoom, scale etc. 
# There is a function ImageDataGenerator which is used here

In [82]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale =1./255,
                                   shear_range =0.25,
                                   zoom_range = 0.25,
                                   horizontal_flip =True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Load the training data

In [83]:
training_set = train_datagen.flow_from_directory('/Users/vaibhavverdhan/Book Writing/DogsCats/train',target_size=(64,64),batch_size= 32,class_mode='binary')

Found 23000 images belonging to 2 classes.


In [ ]:
# Load the testing data

In [84]:
test_set = test_datagen.flow_from_directory('/Users/vaibhavverdhan/Book Writing/DogsCats/test',
                                           target_size = (64,64),
                                           batch_size = 32,
                                           class_mode ='binary')

Found 2000 images belonging to 2 classes.


In [ ]:
# Let us begin the training now. Steps per epoch is 625 and number of epochs is 10. 
#Epoch is one full cycle of the training data
# Steps and Batch size has to be understood next. For example: if we have 1000 images and batch size of 10, it means
# number of steps = 1000/10 which is 100 steps required.
# Depending on the complexity of the network, the number of epochs given etc., the compilation will take time.
# The test dataset is passed as a validation_data here.

In [85]:
from IPython.display import display
from PIL import Image
catDogImageclassifier.fit_generator(training_set,
                        steps_per_epoch =625,
                        epochs = 10,
                        validation_data =test_set,
                        validation_steps = 1000)

Epoch 1/10
625/625 [==============================] - 185s 296ms/step - loss: 0.6721 - acc: 0.5822 - val_loss: 0.6069 - val_acc: 0.6610
Epoch 2/10
625/625 [==============================] - 152s 243ms/step - loss: 0.5960 - acc: 0.6831 - val_loss: 0.5151 - val_acc: 0.7543
Epoch 3/10
625/625 [==============================] - 151s 242ms/step - loss: 0.5452 - acc: 0.7217 - val_loss: 0.4891 - val_acc: 0.7545
Epoch 4/10
625/625 [==============================] - 150s 239ms/step - loss: 0.5069 - acc: 0.7568 - val_loss: 0.4657 - val_acc: 0.7743
Epoch 5/10
625/625 [==============================] - 150s 240ms/step - loss: 0.4813 - acc: 0.7713 - val_loss: 0.4407 - val_acc: 0.7925
Epoch 6/10
625/625 [==============================] - 152s 243ms/step - loss: 0.4526 - acc: 0.7866 - val_loss: 0.4374 - val_acc: 0.7924
Epoch 7/10
625/625 [==============================] - 151s 241ms/step - loss: 0.4458 - acc: 0.7953 - val_loss: 0.3891 - val_acc: 0.8324
Epoch 8/10
625/625 [============================

In [ ]:
# We can see here that in the final epoch we got validation accuracy of 82.21%. 
#We can also see that in Epoch 7 we got accuracy of 83.24 which is better than the final accuarcy.
# There are ways to give a checkpoint in between the training and save that version, 
#we will look at it in subsequent chapters

In [ ]:
# We are saving the final model as a file here. The model can be then loaded again as and when required.
# The model will be saved as a HDF5 file. And it can be reused later.

In [38]:
catDogImageclassifier.save('catdog_cnn_model.h5')

In [ ]:
# Load the saved model. The saved file is loaded using load_model.

In [39]:
from keras.models import load_model 
catDogImageclassifier = load_model('catdog_cnn_model.h5')

In [ ]:
# Check how the model is predicting for an unseen image. 

In [40]:
import numpy as np
from keras.preprocessing import image
an_image =image.load_img('/Users/vaibhavverdhan/Book Writing/2.jpg',target_size =(64,64))# Load the image
# The image is now getting converted to array of numbers
an_image =image.img_to_array(an_image)
#Let us now expand it's dimensions. It will improve the prediction power 
an_image =np.expand_dims(an_image, axis =0)
# call the predict method here
verdict = catDogImageclassifier.predict(an_image)
if verdict[0][0] >= 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'
# Let us print our final prediction    
print(prediction)

dog


In [6]:
# Here in this example, we have designed a Neural Network using Kears. We trained using images of Cats and Dogs. 
#And then tested it.
# It is possible to train a multi-classifier system too. The onus lies to get the images for 
# each of the class.